In [0]:
#Credenciales
sec = dbutils.secrets.get("scpcumpsccrit002","gscp1glbsp4cumpscauth100").split("[###]")
client_id = sec[1]
client_secret = sec[0]
client_endpoint = "https://login.microsoftonline.com/35595a02-4d6d-44ac-99e1-f9ab4cd872db/oauth2/token"
storage_account = "gscp1weustacumpsccrit100"
spark.conf.set(f"fs.azure.account.auth.type.{storage_account}.dfs.core.windows.net", "OAuth")
spark.conf.set(f"fs.azure.account.oauth.provider.type.{storage_account}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set(f"fs.azure.account.oauth2.client.id.{storage_account}.dfs.core.windows.net", client_id)
spark.conf.set(f"fs.azure.account.oauth2.client.secret.{storage_account}.dfs.core.windows.net", client_secret)
spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{storage_account}.dfs.core.windows.net", client_endpoint)


#Capturamos las variables de los widgets
dbfs_dir = dbutils.widgets.get("dbfs_dir")
adls_output_dir = dbutils.widgets.get("adls_output_dir")
date = dbutils.widgets.get("date")

#Nos aseguramos que los ficheros zip y csv asociados a la fecha existen en el directorio
required_files = {f"report_{date}.csv", f"report_{date}.zip"}
files_in_dir = {f.name for f in dbutils.fs.ls(dbfs_dir)}
missing = required_files - files_in_dir
assert(missing == set())

In [0]:
import os
import shutil

#Movemos el zip (descomprimido) y el csv al directorio adls_output_dir/{date}, usando directorios locales intermedios
local_dir = dbfs_dir.replace("dbfs:", "/dbfs")
local_output_dir = f"{local_dir}/output/{date}"

#Aseguramso que no haya nada en el directorio local_output_dir. Si lo hay, habría que eliminarlo, para no duplicar ficheros
os.makedirs(local_output_dir, exist_ok=True)
assert(not os.listdir(local_output_dir))

shutil.unpack_archive(f"{local_dir}/report_{date}.zip", extract_dir=local_output_dir)
shutil.move(f"{local_dir}/report_{date}.csv", f"{local_output_dir}")

dbutils.fs.mv(local_output_dir.replace("/dbfs", "dbfs:"), f"{adls_output_dir}/{date}", recurse=True)

dbutils.fs.ls(f"{adls_output_dir}/{date}")